In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../scripts")
import util
import camera
from track import Track
from scipy.optimize import minimize

np.set_printoptions(precision=4)
!pwd

/home/zpyang/git/visnet/src/visnet/notebooks


In [2]:
def target_traj_circle(t):
    x = 15*np.sin(2 * np.pi * 0.01 * t / 3)
    y = 10*np.cos(2 * np.pi * 0.01 * t / 3)
    z = 20
    return np.array([x,y,z])

def target_traj_straight(t, start, finish, tf):
    max_dist = np.linalg.norm(start-finish)
    v_max = (finish-start) / tf

    return start + v_max * t

def target_stationary(start):
    return start

In [3]:
# %%timeit
cam_param = [642.0926, 642.0926, 1000.5, 1000.5,0]
# [x y z roll pitch yaw]
cam_poses = np.array([
    [20, 20, 12, 0, 0, -2.2],
    [20, -20, 12, 0, 0, 2.2],
    [-20, -20, 12, 0, 0, 0.7],
    [-20, 20, 12, 0, 0, -0.7],
])

cam_group = camera.CamGroup(cam_param, cam_poses[0:4,:])

n_targets = 2
tracks = []

t_final = 200

verbose = False
n_particles = 1000

x0_1 = np.array([-20, 3, 17])
# x0_1 = np.array([0, 10, 20])
# track1 = Track(n_particles, x0=x0_1, label=1)


track1 = Track(n_particles, x0=None, label=1)

tracks = [track1]
tracks_true = x0_1

state_est1 = []



cov_hist = []

for t in range(t_final):
    target_1 = target_traj_straight(t, x0_1, x0_1+[40,0,0], t_final)

    # target_1 = target_traj_circle(t)

    targets = np.vstack([target_1])
    tracks_true = np.vstack([tracks_true, targets])

    #each camera can give multiple measurements
    z = cam_group.get_group_measurement(targets, labels= np.array([1]))

    mean_states = np.array([track.mean_state[0:3] for track in tracks])  
    mean_hypo = cam_group.get_group_measurement(mean_states, labels=np.array([1]))

    z_a = []
    for z_m, mean_hypo_m in zip(z, mean_hypo):
        rand = np.random.uniform(0,1)
        z_a_m, order = util.msmt_association(z_m, mean_hypo_m, sigma=20)
        
        z_a.append(z_a_m)

    for i, track in enumerate(tracks):
        msmt = [z_a_m[i] for z_a_m in z_a]
        particles = util.dynamics_d(track.particles)
        hypo = util.observe_fn(cam_group, track.particles)
        weights = util.weight_fn(msmt=msmt, hypo=hypo, sigma=30)
        
        track.update(weights, particles)
        cov_hist.append(track.cov_state)


In [5]:
plt.rcParams.update({'font.size': 18})
traj_1 = np.array(track1.trajectory)

fig2 = plt.figure("Association accuracy and Track Error", figsize=(20,10))

error = traj_1-tracks_true[0:-1, 0:3]
std = np.std(error[80:,:], axis=0)
std_cam = np.sqrt(2*20**2) * 20/642
print(std, std_cam)

ax = plt.axes()
ax.plot(traj_1-tracks_true[0:-1, 0:3], label=["x","y","z"])
ax.set_xlabel("frames")
ax.set_ylabel("position error[m]")
ax.grid()
ax.legend()
plt.title("Tracking Error of a Single Target");



# plt.suptitle("Maneuvering Target")

[0.1361 0.1351 0.1106] 0.8811299454037975


In [100]:
x


In [26]:
fig = plt.figure("trajectory", figsize=(10,10))

ax = plt.axes(projection="3d")

ax.plot3D(traj_1[:,0], traj_1[:,1], traj_1[:,2], marker='.', color='b')
ax.plot3D(tracks_true[:, 0], tracks_true[:, 1], tracks_true[:, 2], color='y')


ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)
ax.set_xlabel('x[m]')
ax.set_ylabel('y[m]')
ax.set_zlabel('z[m]')

Text(0.5, 0, 'z[m]')

In [8]:
fig = plt.figure("trajectory-2d", figsize=(10,10))

ax = plt.axes()
traj_1 = np.array(track1.trajectory)
ax.plot(traj_1[:,0], traj_1[:,1], marker='.', color='b', label="track 1")
ax.plot(tracks_true[:, 0], tracks_true[:, 1], color='y', label="track 1 ground truth")
ax.set_xlim(-25,25)
ax.set_ylim(-25,25)
ax.set_xlabel('x[m]')
ax.set_ylabel('y[m]')
ax.grid()
ax.legend()
ax.set_title("Top-Down View of Trajectories of the Target")


Text(0.5, 1.0, 'Top-Down View of Trajectories of the Target')

In [22]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import cm
# Attaching 3D axis to the figure
fig = plt.figure("Particle visualization", figsize=(10, 10))
# ax1 = plt.axes(projection="3d")
# ax1.view_init(90,-90)
ax1 = plt.axes()
# ax1 = fig.add_subplot(1, 2, 1, projection="3d")
# ax2 = fig.add_subplot(1, 2, 2, projection="3d")


particles_1 = np.array(track1.particles_hist)

weights_1 = np.array(track1.weights_hist)

print(particles_1.shape, weights_1.shape)
def update_frame(i):
    # NOTE: there is no .set_data() for 3 dim data...
    x_1 = particles_1[i, :, 0]
    y_1 = particles_1[i, :, 1]
    z_1 = particles_1[i, :, 2]
    w_1 = weights_1[i, :]

    
    ax1.cla()
    ax1.set_xlim(-25,25)
    ax1.set_ylim(-25,25)
    ax1.scatter(x_1, y_1, cmap=cm.turbo, c=w_1, s=2)
    # ax1.set_xlim3d(-25,25)
    # ax1.set_ylim3d(-25,25)
    # ax1.set_zlim3d(0,25)
    # ax1.scatter(x_1, y_1, z_1, cmap=cm.turbo, c=w_1, s=2)
    ax1.set_xlabel('x, m')
    ax1.set_ylabel('y, m')
    ax1.grid()
    # ax1.set_zlabel('z, m')

    # if i == 100:
    #     plt.savefig("../plots/frame_{}".format(i))
        
    return None


ani = FuncAnimation(
    fig, update_frame, t_final, fargs=[],
    interval=int(1000 / 10), blit=False)
# plt.close()
# ani.save("../videos/particles.mp4")
# HTML(ani.to_html5_video())
# plt.close()

(300, 1000, 7) (300, 1000)
